# Momentum Strategy - Weekly Rebalance


1. Rank Stocks in SP500 based on momentum. Momentum: calculated by multiplying the annualized exponential regression slope of past 90 days by R^2 coeeffecient of regression calc
2. Position Size: Calc using 20 day ATR of each stock, multiplied by 10 basis points of portfolio value
3. Open new positions only if SP 500 is above 200 day moving avg
4. Every week, sell stocks that are not in top 20% momentum ranking, or have fallen below their 100 day moving avg. Buy stocks in top 20% momentum rankings
5. Every other week, rebalance exisitng positions with updated ATR values
https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/


Alpaca / Synthetic long short :
https://www.theoptionsguide.com/synthetic-short-stock.aspx
https://www.theoptionsguide.com/synthetic-long-stock.aspx?source=post_page-----25d580ccab0c----------------------
https://www.investopedia.com/articles/optioninvestor/08/synthetic-options.asp

https://medium.com/automation-generation/ultimate-list-of-automated-trading-strategies-you-should-know-part-3-25d580ccab0c

DATA: 
* AlphaVantage: https://www.alphavantage.co/documentation/6a772cd25
* QUANDL
* Or just learn Quant Connect
* Look into Kelly Edge

### To Run
* Run get_data.py from terminal
* Run this notebook
* To change % Thresholds, edit trainModel.py

In [1]:
import requests
import pandas as pd
import os
import bs4 as bs
import numpy as np
import trainModel
from scipy.stats import linregress

In [2]:
def get_sp500():
    url = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    resp = requests.get(url)
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)  
    #with open("sp500tickers.pickle","wb") as f:
    #    pickle.dump(tickers,f)  
    return tickers

In [18]:
def generate_massive_df(tickers):
    stocks = (
        (pd.concat(
            [pd.read_csv(f"data/{ticker}.csv", index_col='Date', parse_dates=True)[
                'Close'
            ].rename(ticker)
            for ticker in tickers if os.path.exists("data/{}.csv".format(ticker))],
            axis=1,
            sort=True)
        )
    )
    stocks = stocks.loc[:,~stocks.columns.duplicated()]
    return stocks

In [7]:
def compile_data():
    tickers = get_sp500()
    main_df = pd.DataFrame()
    for num, ticker in enumerate(tickers):
        if not os.path.exists("data/{}.csv".format(ticker)):
            continue
        df = pd.read_csv("data/{}.csv".format(ticker))
        df.set_index('Date', inplace=True)
        df.rename(columns = {'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')
            
        if num % 10 == 0:
            print("{} csvs done".format(num))
        
    #print(main_df.head())
    main_df.to_csv('data/joined/sp500_joined_closes.csv')

In [27]:
def train_all(tickers):
    """
    Returns long list and short list (top 10 accuracy buys and top 10 accuracy sells)
    """
    long = []
    short = []
    hold = []
    for count,ticker in enumerate(tickers):
        if count % 5 == 0:
            print("training {} out of 500".format(count))
        bsh, acc = trainModel.train_test('AAPL')
        if bsh == 1:
            long.append((ticker, bsh,acc))
        elif bsh == -1:
            short.append((ticker,bsh,acc))
        else:
            hold.append((ticker,bsh,acc))
        
    #sorted(long,key=lambda x: x[2], reverse=True)
    #sorted(short,key=lambda x: x[2], reverse=True)
    #sorted(hold,key=lambda x: x[2], reverse=True)
        
    #return long[:10], short[:10], hold[:10]
    return long, short, hold
    
    

In [28]:
tickers = get_sp500()
compile_data()
long, short, hold = train_all(tickers)

0 csvs done
10 csvs done
20 csvs done
30 csvs done
40 csvs done
50 csvs done
60 csvs done
70 csvs done
80 csvs done
90 csvs done
100 csvs done
110 csvs done
120 csvs done
130 csvs done
140 csvs done
150 csvs done
160 csvs done
170 csvs done
180 csvs done
190 csvs done
200 csvs done
210 csvs done
220 csvs done
230 csvs done
240 csvs done
250 csvs done
260 csvs done
270 csvs done
280 csvs done
290 csvs done
300 csvs done
310 csvs done
320 csvs done
330 csvs done
340 csvs done
350 csvs done
360 csvs done
370 csvs done
380 csvs done
390 csvs done
400 csvs done
410 csvs done
420 csvs done
430 csvs done
440 csvs done
450 csvs done
460 csvs done
470 csvs done
480 csvs done
490 csvs done
500 csvs done
training 0 out of 500
Data spread:  Counter({'1': 586, '0': 393, '-1': 225})


/home/seb/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Data spread:  Counter({'1': 586, '0': 393, '-1': 225})


/home/seb/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Data spread:  Counter({'1': 586, '0': 393, '-1': 225})


...

/home/seb/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Data spread:  Counter({'1': 586, '0': 393, '-1': 225})


/home/seb/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Data spread:  Counter({'1': 586, '0': 393, '-1': 225})


/home/seb/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [36]:
long = sorted(long,key=lambda x: x[2], reverse=True)
short = sorted(short,key=lambda x: x[2], reverse=True)
hold = sorted(hold,key=lambda x: x[2], reverse=True)

In [37]:
long[:10]

[('AES', 1, 0.521594684385382),
 ('DFS', 1, 0.5016611295681063),
 ('FLT', 1, 0.4983388704318937),
 ('KSS', 1, 0.4983388704318937),
 ('ANET', 1, 0.4950166112956811),
 ('LH', 1, 0.4950166112956811),
 ('NVR', 1, 0.4950166112956811),
 ('PNR', 1, 0.4950166112956811),
 ('LW', 1, 0.4883720930232558),
 ('TSN', 1, 0.4883720930232558)]

In [38]:
short[:10]

[('LOW', -1, 0.4717607973421927),
 ('BAC', -1, 0.4684385382059801),
 ('JNPR', -1, 0.4684385382059801),
 ('SBAC', -1, 0.46511627906976744),
 ('FITB', -1, 0.46179401993355484),
 ('IPGP', -1, 0.46179401993355484),
 ('AVY', -1, 0.45514950166112955),
 ('KEYS', -1, 0.45514950166112955),
 ('FTI', -1, 0.45182724252491696),
 ('PNC', -1, 0.4485049833887043)]

In [39]:
hold[:10]

[('SO', 0, 0.5116279069767442),
 ('TMO', 0, 0.4950166112956811),
 ('LNT', 0, 0.4883720930232558),
 ('CELG', 0, 0.4850498338870432),
 ('MA', 0, 0.48172757475083056),
 ('NSC', 0, 0.47840531561461797),
 ('WMB', 0, 0.47840531561461797),
 ('GM', 0, 0.4750830564784053),
 ('STX', 0, 0.4750830564784053),
 ('CAH', 0, 0.4684385382059801)]

# To Trade
* Only buy if SPY is above 200 day moving avg
* Sell stock if it is below its 100 day moving avg, or not in buy list anymore
* Rebalance every 2 weeks with ATR values
* Position Size = (Account Value * risk parity) / ATR20

In [21]:
def calc_size(atr, risk_factor=0.05, val=25000):
    return (val * risk_factor) / atr

In [47]:
calc_size(2.23)

560.5381165919282

# Curr Portfolio (25000 ToS paper)
* AES: 244 shares
* KSS: 14 shares
* PNR: 40 shares
* LW: 8 shares
* TSN: 6 shares
* LOW: 1 put


#PL from Mon Oct 14th (all end of day):
* Mon: